# Partition spots into cell segmentation

In [1]:
# Need to install ImageAnalysis3 and run the appropriate Startup_py3.py
%run "C:\Users\cosmosyw\Documents\Softwares\ImageAnalysis3\required_files\Startup_py3.py"
# append path to import ImageAnalysis3
sys.path.append(r"C:\Users\cosmosyw\Documents\Softwares")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

23192


## 0.1 Folders

In [2]:
save_folder = r'E:\DNA_analysis\20220819_Mecp2_DNA_from_0809'
pre_process_folder = r'Z:\Cosmos_DNA\MeC2_0809_DNA_preprocessed'

save_filenames = [os.path.join(pre_process_folder, _fl) for _fl in os.listdir(pre_process_folder)
                  if _fl.split(os.extsep)[-1]=='hdf5']
# extract fov_id
save_fov_ids = [int(os.path.basename(_fl).split('.hdf5')[0].split('_')[-1]) for _fl in save_filenames]

debug = False

print(f"{len(save_filenames)} fovs detected")

analysis_folder = os.path.join(save_folder, 'Analysis_0508')

segmentation_folder = os.path.join(analysis_folder, 'Segmentation')
if not os.path.exists(segmentation_folder):
    os.makedirs(segmentation_folder)
    print(f"Creating segmentation_folder: {segmentation_folder}")
else:
    print(f"Use segmentation_folder: {segmentation_folder}")

cand_spot_folder = os.path.join(analysis_folder, 'CandSpots')
if not os.path.exists(cand_spot_folder):
    os.makedirs(cand_spot_folder)
    print(f"Creating cand_spot_folder: {cand_spot_folder}")
else:
    print(f"Use cand_spot_folder: {cand_spot_folder}")

decoder_folder = cand_spot_folder.replace('CandSpots', 'Decoder')
if debug:
    _version = 0
    while os.path.exists(os.path.join(decoder_folder, f'v{_version}')):
        _version += 1
    decoder_folder = os.path.join(decoder_folder, f'v{_version}')
if not os.path.exists(decoder_folder):
    os.makedirs(decoder_folder)
    print(f"Creating decoder_folder: {decoder_folder}")
else:
    print(f"Use decoder_folder: {decoder_folder}")
    
picked_folder = os.path.join(analysis_folder, 'Picked_v0')
if not os.path.exists(picked_folder):
    os.makedirs(picked_folder)
    print(f"Creating picked_folder: {picked_folder}")
else:
    print(f"Use picked_folder: {picked_folder}")

243 fovs detected
Use segmentation_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation
Use cand_spot_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\CandSpots
Use decoder_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Decoder
Use picked_folder: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Picked_v0


In [3]:
save_fov_ids = save_fov_ids[:] #finish procesing the remaining
save_filenames = save_filenames[:]

In [4]:
pixel_sizes = np.array([250,108,108])
single_im_size = np.array([50,2048,2048])

# 1. Partition DNA-MERFISH spots

In [5]:
from ImageAnalysis3.classes.partition_spots import Spots_Partition
from ImageAnalysis3.classes.preprocess import Spots3D
from ImageAnalysis3.figure_tools import plot_partition
import pandas as pd

In [6]:
from ImageAnalysis3.segmentation_tools.cell import Align_Segmentation
import ImageAnalysis3.io_tools.spots 

In [7]:
search_radius = 9
overwrite_cand_spots = False
add_relabel_spots = False
data_type = 'combo' # 'combo'
#_total_seg_save_file defined in 2.3
if not os.path.exists(os.path.join(cand_spot_folder, data_type+'_original')):
    print(os.path.join(cand_spot_folder, data_type+'_original'))
    os.makedirs(os.path.join(cand_spot_folder, data_type+'_original'))
# load microscope.json
dna_microscope_file = r'\\mendel\pu_documents\Merfish_analysis\Merfish_Analysis_Scripts\merlin_parameters\microscope\storm6_microscope.json'

In [ ]:
%%time
_partition_args = []

for _fov_id, _save_filename in zip(save_fov_ids, save_filenames):

    _segmentation_savefile = os.path.join(segmentation_folder, 
        os.path.basename(_save_filename).replace('.hdf5', '_Segmentation.hdf5') )
    # savename
    _cand_spot_filename = os.path.join(cand_spot_folder, data_type+'_original', 
                                      os.path.basename(_save_filename).replace('.hdf5', f'_CandSpots.csv') )
    if os.path.isfile(_cand_spot_filename) and not overwrite_cand_spots:
        continue
    # load segmentation label matrix and uids
    _align_seg = Align_Segmentation('', '', _save_filename, '', dna_microscope_file, np.array([]))
    _align_seg._load(_segmentation_savefile)
    seg_label, fovcell_2_uid = _align_seg.dna_mask, _align_seg.fovcell_2_uid
    # load spots
    spots_list, combo_bits = ia.io_tools.spots.load_preprocess_spots(
        _save_filename, data_type, None, pixel_sizes=pixel_sizes,
    )
    # add relabeled spots if specified.
    if add_relabel_spots:
        relabel_spots_list, relabel_bits = ia.io_tools.spots.load_preprocess_spots(
            _save_filename, f'relabeled_{data_type}', None, pixel_sizes=pixel_sizes,
        )
        for _rspots, _rbit in zip(relabel_spots_list, relabel_bits):
            _ind = list(combo_bits).index(_rbit)
            _old_spots = spots_list[_ind]
            # merge
            _merged_spots = ia.io_tools.spots.merge_RelabelSpots(
                _old_spots, _rspots, pixel_sizes=pixel_sizes,
            )
            # replace
            spots_list[_ind] = _merged_spots
    # 
    _all_spots = ia.io_tools.spots.merge_Spots3DList(spots_list, pixel_sizes=pixel_sizes)
    # partition args
    _args = (
        _fov_id, 
        np.array(_all_spots), _all_spots.bits, _all_spots.channels,
        seg_label, fovcell_2_uid,
        dna_microscope_file,
        search_radius, pixel_sizes,
        True, 
        True, _cand_spot_filename,
        True, 60, 
        False, True,
    )
    _partition_args.append(_args)
    
print(len(_partition_args))

## parallel process to partition spots

In [ ]:
%%time
import multiprocessing as mp
# Multiprocessing
print(f"- Start multiprocessing spot partitioning", end=' ')
_start_time = time.time()
with mp.Pool(6) as _partition_pool:
    # start multiprocessing
    _partition_pool.starmap(ia.classes.partition_spots.batch_partition_DNA_spots, 
                            _partition_args, chunksize=1)
    # close multiprocessing
    _partition_pool.close()
    _partition_pool.join()
    _partition_pool.terminate()
print(f"finish in {time.time()-_start_time:.3f}s. ")

## Filter candspots based on RNA

Filter out all cells with undetermined genotypes for Mecp2
So further analysis only focuses on cells with defined genotypes.

In [13]:
#################################### MANUAL input #############################################
sel_exp = 809

# this dataframe contains the cell information from this experiment
df_exp = pd.read_csv(r"C:\Users\cosmosyw\Documents\Research\Mecp2_v2\results\MeCP2_antibody\MeCP2_0809_antibody.csv",
                    index_col=0)
#################################### MANUAL input #############################################

# this dataframe contains all information regarding the genotypes and cell types of cells
df_cells = pd.read_csv(r"C:\Users\cosmosyw\Documents\Research\Mecp2_v2\results\cell_composition\df_cell_with_MeCP2_genotype.csv",
                      index_col=0)

kept_uids = df_cells[df_cells['experiment']==sel_exp]['uid'].values

df_exp = df_exp[df_exp['uid'].isin(kept_uids)].copy()

kept_cellIDs_each_fov = {}
for fov, df_fov in df_exp.groupby('fov'):
    kept_cellIDs_each_fov[fov] = df_fov.cellID.values

In [ ]:
input_folder = os.path.join(cand_spot_folder, 'combo_original')

output_folder = os.path.join(cand_spot_folder, 'combo')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
from tqdm import tqdm

for file in tqdm(os.listdir(input_folder)):
    if '.csv' in file:
        output_file = os.path.join(output_folder, file)
        if os.path.exists(output_file):
            continue
        
        df = pd.read_csv(os.path.join(input_folder, file))
        fov = df['fov_id'].values[0]
        if fov in kept_cellIDs_each_fov.keys():
            kept_cellids = kept_cellIDs_each_fov[fov]
        else:
            kept_cellids = []
        df = df[df['cell_id'].isin(kept_cellids)].copy()
        df.to_csv(output_file)

# Partition sequential genes

In [8]:
search_radius = 9
overwrite_cand_spots = False
add_relabel_spots = False
data_type = 'unique' # 'combo'
#_total_seg_save_file defined in 2.3
if not os.path.exists(os.path.join(cand_spot_folder, data_type+'_original')):
    print(os.path.join(cand_spot_folder, data_type+'_original'))
    os.makedirs(os.path.join(cand_spot_folder, data_type+'_original'))
# load microscope.json
dna_microscope_file = r'\\mendel\pu_documents\Merfish_analysis\Merfish_Analysis_Scripts\merlin_parameters\microscope\storm6_microscope.json'

In [9]:
%%time
_partition_args = []

for _fov_id, _save_filename in zip(save_fov_ids, save_filenames):

    _segmentation_savefile = os.path.join(segmentation_folder, 
        os.path.basename(_save_filename).replace('.hdf5', '_Segmentation.hdf5') )
    # savename
    _cand_spot_filename = os.path.join(cand_spot_folder, data_type+'_original', 
                                      os.path.basename(_save_filename).replace('.hdf5', f'_CandSpots.csv') )
    if os.path.isfile(_cand_spot_filename) and not overwrite_cand_spots:
        continue
    # load segmentation label matrix and uids
    _align_seg = Align_Segmentation('', '', _save_filename, '', dna_microscope_file, np.array([]))
    _align_seg._load(_segmentation_savefile)
    seg_label, fovcell_2_uid = _align_seg.dna_mask, _align_seg.fovcell_2_uid
    # load spots
    spots_list, combo_bits = ia.io_tools.spots.load_preprocess_spots(
        _save_filename, data_type, None, pixel_sizes=pixel_sizes,
    )
    # add relabeled spots if specified.
    if add_relabel_spots:
        relabel_spots_list, relabel_bits = ia.io_tools.spots.load_preprocess_spots(
            _save_filename, f'relabeled_{data_type}', None, pixel_sizes=pixel_sizes,
        )
        for _rspots, _rbit in zip(relabel_spots_list, relabel_bits):
            _ind = list(combo_bits).index(_rbit)
            _old_spots = spots_list[_ind]
            # merge
            _merged_spots = ia.io_tools.spots.merge_RelabelSpots(
                _old_spots, _rspots, pixel_sizes=pixel_sizes,
            )
            # replace
            spots_list[_ind] = _merged_spots
    # 
    _all_spots = ia.io_tools.spots.merge_Spots3DList(spots_list, pixel_sizes=pixel_sizes)
    # partition args
    _args = (
        _fov_id, 
        np.array(_all_spots), _all_spots.bits, _all_spots.channels,
        seg_label, fovcell_2_uid,
        dna_microscope_file,
        search_radius, pixel_sizes,
        True, 
        True, _cand_spot_filename,
        True, 4, 
        False, True,
    )
    _partition_args.append(_args)
    
print(len(_partition_args))

-- loading segmentation info from fov:1 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_001_Segmentation.hdf5
-- loading segmentation info from fov:2 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_002_Segmentation.hdf5
-- loading segmentation info from fov:3 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_003_Segmentation.hdf5
-- loading segmentation info from fov:4 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_004_Segmentation.hdf5
-- loading segmentation info from fov:5 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_005_Segmentation.hdf5
-- loading segmentation info from fov:6 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_006_Segmentation.hdf5
-- loading segmentation info from fov:7 into file: E:\DNA_analys

-- loading segmentation info from fov:54 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_054_Segmentation.hdf5
-- loading segmentation info from fov:55 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_055_Segmentation.hdf5
-- loading segmentation info from fov:56 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_056_Segmentation.hdf5
-- loading segmentation info from fov:57 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_057_Segmentation.hdf5
-- loading segmentation info from fov:58 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_058_Segmentation.hdf5
-- loading segmentation info from fov:59 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_059_Segmentation.hdf5
-- loading segmentation info from fov:60 into file: E:\DNA

-- loading segmentation info from fov:110 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_110_Segmentation.hdf5
-- loading segmentation info from fov:111 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_111_Segmentation.hdf5
-- loading segmentation info from fov:112 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_112_Segmentation.hdf5
-- loading segmentation info from fov:113 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_113_Segmentation.hdf5
-- loading segmentation info from fov:114 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_114_Segmentation.hdf5
-- loading segmentation info from fov:115 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_115_Segmentation.hdf5
-- loading segmentation info from fov:116 into file:

-- loading segmentation info from fov:165 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_165_Segmentation.hdf5
-- loading segmentation info from fov:166 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_166_Segmentation.hdf5
-- loading segmentation info from fov:167 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_167_Segmentation.hdf5
-- loading segmentation info from fov:168 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_168_Segmentation.hdf5
-- loading segmentation info from fov:169 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_169_Segmentation.hdf5
-- loading segmentation info from fov:170 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_170_Segmentation.hdf5
-- loading segmentation info from fov:171 into file:

-- loading segmentation info from fov:218 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_218_Segmentation.hdf5
-- loading segmentation info from fov:219 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_219_Segmentation.hdf5
-- loading segmentation info from fov:220 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_220_Segmentation.hdf5
-- loading segmentation info from fov:221 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_221_Segmentation.hdf5
-- loading segmentation info from fov:222 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_222_Segmentation.hdf5
-- loading segmentation info from fov:223 into file: E:\DNA_analysis\20220819_Mecp2_DNA_from_0809\Analysis_0508\Segmentation\Conv_zscan_223_Segmentation.hdf5
-- loading segmentation info from fov:224 into file:

In [10]:
%%time
import multiprocessing as mp
# Multiprocessing
print(f"- Start multiprocessing spot partitioning", end=' ')
_start_time = time.time()
with mp.Pool(6) as _partition_pool:
    # start multiprocessing
    _partition_pool.starmap(ia.classes.partition_spots.batch_partition_DNA_spots, 
                            _partition_args, chunksize=1)
    # close multiprocessing
    _partition_pool.close()
    _partition_pool.join()
    _partition_pool.terminate()
print(f"finish in {time.time()-_start_time:.3f}s. ")

- Start multiprocessing spot partitioning finish in 395.638s. 
CPU times: total: 1min 45s
Wall time: 6min 35s


In [11]:
input_folder = os.path.join(cand_spot_folder, 'unique_original')

output_folder = os.path.join(cand_spot_folder, 'unique')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [14]:
from tqdm import tqdm

for file in tqdm(os.listdir(input_folder)):
    if '.csv' in file:
        output_file = os.path.join(output_folder, file)
        if os.path.exists(output_file):
            continue
        
        df = pd.read_csv(os.path.join(input_folder, file))
        
        df = df[df['height']>=1000].copy()
        df = df[df['sigma_x']<=3].copy()
        df = df[df['sigma_y']<=3].copy()
        df = df[df['sigma_z']<=3].copy()
        
        if len(df)==0:
            df.to_csv(output_file)
            continue
            
        fov = df['fov_id'].values[0]
        if fov in kept_cellIDs_each_fov.keys():
            kept_cellids = kept_cellIDs_each_fov[fov]
        else:
            kept_cellids = []
        df = df[df['cell_id'].isin(kept_cellids)].copy()
        df.to_csv(output_file)

100%|███████████████████████████████████████████████████████████████████████████████| 244/244 [00:22<00:00, 10.94it/s]
